In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [14]:
# Telecharger les données 'All transactions for a randomly selected sample of 50,000 customers'
# sur https://www.dunnhumby.com/source-files/
my_path = "D:/data_advanced_ML/dunnhumby_Let's-Get-Sort-of-Real-(Sample-50K-customers)/dunnhumby_Let's-Get-Sort-of-Real-(Sample-50K-customers)/"

In [15]:
def load_data_new(my_path = my_path):
    list_df = []
    for file in os.listdir(my_path.replace("\\","/")):
        if file.startswith('tr'):
            df = pd.read_csv(my_path + file)
            cols = [col for col in df.columns if col in ['SHOP_WEEK', 'QUANTITY', 'SPEND', 'PROD_CODE', 'STORE_CODE']]
            list_df.append(df[cols])

    df = pd.concat(list_df,axis=0)
    return df

df = load_data_new()

In [16]:
df.columns

Index(['SHOP_WEEK', 'QUANTITY', 'PROD_CODE', 'STORE_CODE', 'SPEND'], dtype='object')

In [17]:
df.head()

,SHOP_WEEK,QUANTITY,PROD_CODE,STORE_CODE,SPEND
0,200607,1,PRD0900011,STORE00001,NaN
1,200607,3,PRD0900035,STORE00001,NaN
2,200607,1,PRD0900043,STORE00001,NaN
3,200607,1,PRD0900057,STORE00001,NaN
4,200607,1,PRD0900058,STORE00001,NaN


In [18]:
spend_na = df['SPEND'].isna()
df = df.loc[~spend_na]

In [19]:
#freq_df = df.groupby(['PROD_CODE', 'BASKET_TYFN']).size().reset_index(name='frequency')

In [20]:
#df = df[['SHOP_WEEK', 'QUANTITY', 'SPEND', 'PROD_CODE', 'STORE_CODE']]

In [21]:
print('Nombre de produits uniques:', len(df['PROD_CODE'].unique()))
nb_products = 1000
top_products = df['PROD_CODE'].value_counts().head(nb_products).index.tolist()
df_top = df[df['PROD_CODE'].isin(top_products)]

Nombre de produits uniques: 4997


In [22]:
print('Nombre de magasins uniques:', len(df['STORE_CODE'].unique()))
df = df[df['QUANTITY'] > 0].copy()
#On ne prend que les magasins avec le plus de transactions
nb_stores = 50
top_stores = df_top['STORE_CODE'].value_counts().head(nb_stores).index.tolist()
df_top = df_top[df_top['STORE_CODE'].isin(top_stores)]

Nombre de magasins uniques: 761


In [23]:
#On prend le prix unitaire par transaction
df_top['UNIT_PRICE'] = df_top['SPEND'] / df_top['QUANTITY']

In [24]:
df_final = df_top.pivot_table(
    index=['SHOP_WEEK', 'STORE_CODE'],
    columns='PROD_CODE',
    values='UNIT_PRICE',
    aggfunc='mean'
)
df_final.reset_index(inplace=True)

In [25]:
df_final

PROD_CODE,SHOP_WEEK,STORE_CODE,PRD0900005,PRD0900006,PRD0900008,PRD0900013,PRD0900014,PRD0900015,PRD0900017,PRD0900027,...,PRD0904961,PRD0904962,PRD0904963,PRD0904967,PRD0904976,PRD0904977,PRD0904979,PRD0904981,PRD0904988,PRD0904994
0,200608,STORE00024,1.01,NaN,1.77,NaN,NaN,NaN,NaN,NaN,...,NaN,2.09,NaN,NaN,1.49,NaN,NaN,0.41,0.28,NaN
1,200608,STORE00065,NaN,NaN,NaN,NaN,0.92,2.210,1.99,NaN,...,NaN,2.09,NaN,NaN,1.49,NaN,NaN,NaN,NaN,NaN
2,200608,STORE00276,1.01,NaN,1.77,NaN,0.92,NaN,NaN,1.87,...,NaN,2.09,NaN,NaN,1.49,NaN,NaN,0.41,0.28,NaN
3,200608,STORE00277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.32,2.09,0.81,NaN,1.49,NaN,0.31,0.41,0.28,NaN
4,200608,STORE00278,NaN,NaN,NaN,NaN,NaN,1.035,NaN,1.87,...,NaN,2.09,NaN,NaN,1.49,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783,200819,STORE02577,1.01,0.22,1.77,NaN,NaN,NaN,NaN,NaN,...,0.32,2.09,NaN,NaN,1.49,1.11,NaN,NaN,0.28,NaN
5784,200819,STORE02614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.09,NaN,NaN,1.49,NaN,NaN,NaN,0.28,NaN
5785,200819,STORE02746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.32,2.09,0.81,NaN,1.49,NaN,NaN,NaN,0.28,1.06
5786,200819,STORE02797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.09,NaN,NaN,1.49,NaN,NaN,NaN,0.28,NaN


In [26]:
# Nombre de semain sans achat pour chaque produit dans chaque magasin
df_final.isnull().sum()/len(df_final) * 100

PROD_CODE
SHOP_WEEK      0.000000
STORE_CODE     0.000000
PRD0900005    85.400829
PRD0900006    75.051831
PRD0900008    50.812025
                ...    
PRD0904977    74.602626
PRD0904979    82.446441
PRD0904981    81.409813
PRD0904988    43.140981
PRD0904994    81.945404
Length: 1002, dtype: float64

In [27]:
df_final

PROD_CODE,SHOP_WEEK,STORE_CODE,PRD0900005,PRD0900006,PRD0900008,PRD0900013,PRD0900014,PRD0900015,PRD0900017,PRD0900027,...,PRD0904961,PRD0904962,PRD0904963,PRD0904967,PRD0904976,PRD0904977,PRD0904979,PRD0904981,PRD0904988,PRD0904994
0,200608,STORE00024,1.01,NaN,1.77,NaN,NaN,NaN,NaN,NaN,...,NaN,2.09,NaN,NaN,1.49,NaN,NaN,0.41,0.28,NaN
1,200608,STORE00065,NaN,NaN,NaN,NaN,0.92,2.210,1.99,NaN,...,NaN,2.09,NaN,NaN,1.49,NaN,NaN,NaN,NaN,NaN
2,200608,STORE00276,1.01,NaN,1.77,NaN,0.92,NaN,NaN,1.87,...,NaN,2.09,NaN,NaN,1.49,NaN,NaN,0.41,0.28,NaN
3,200608,STORE00277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.32,2.09,0.81,NaN,1.49,NaN,0.31,0.41,0.28,NaN
4,200608,STORE00278,NaN,NaN,NaN,NaN,NaN,1.035,NaN,1.87,...,NaN,2.09,NaN,NaN,1.49,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783,200819,STORE02577,1.01,0.22,1.77,NaN,NaN,NaN,NaN,NaN,...,0.32,2.09,NaN,NaN,1.49,1.11,NaN,NaN,0.28,NaN
5784,200819,STORE02614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.09,NaN,NaN,1.49,NaN,NaN,NaN,0.28,NaN
5785,200819,STORE02746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.32,2.09,0.81,NaN,1.49,NaN,NaN,NaN,0.28,1.06
5786,200819,STORE02797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.09,NaN,NaN,1.49,NaN,NaN,NaN,0.28,NaN


In [29]:
df_final.to_csv(os.path.join('data', 'data_top_1000_product_top_50_stores.csv'),index = False)